# CLIXO Tree builder

Cleanup tree data created with Cytoscape and link anntation from mapping file.

In [4]:
!wget "https://gist.githubusercontent.com/keiono/2b2e289371b7aff1f47d5e2c2a41fc2a/raw/37bb6d2bd1d81072f95721fd83d88ac3774e365c/clixo-final.cyjs"  -O ./data/clixo-final.cyjs

--2017-01-09 16:08:19--  https://gist.githubusercontent.com/keiono/2b2e289371b7aff1f47d5e2c2a41fc2a/raw/37bb6d2bd1d81072f95721fd83d88ac3774e365c/clixo-final.cyjs
Resolving gist.githubusercontent.com... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to gist.githubusercontent.com|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5726161 (5.5M) [text/plain]
Saving to: ‘./data/clixo-final.cyjs’

./data/clixo-final. 100%[===================>]   5.46M  21.2MB/s    in 0.3s    

2017-01-09 16:08:20 (21.2 MB/s) - ‘./data/clixo-final.cyjs’ saved [5726161/5726161]



In [2]:
import json
with open('./data/clixo-mapping.json', 'r') as f:
    clixo_map = json.load(f)

In [5]:
with open('./data/clixo-final.cyjs', 'r') as f:
    clixo = json.load(f)

In [17]:
from goatools import obo_parser

# Annotation file for the CLIXO terms
oboUrl = './data/go.obo'

obo = obo_parser.GODag(oboUrl, optional_attrs=['def'])

load obo file ./data/go.obo
./data/go.obo: fmt(1.2) rel(2017-01-08) 47,914 GO Terms


In [30]:
nodes = clixo['elements']['nodes']
new_nodes = []

id2clixo = {}
bet = []

for n in nodes:
    d = n['data']
    id2clixo[d['id']] = d['id_original']
    clixo_id = d['id_original'].split(':')[1]
    name = d['id_original']
    
    if clixo_id in clixo_map.keys():
        goid = clixo_map[clixo_id]['go']
        name = obo[goid].name
    
    bet.append(d['BetweennessCentrality'])
    
    data = {
        'id': d['id_original'],
        'name': name,
        'betweenness': d['BetweennessCentrality']
    }
    
    new_nodes.append({
            'data': data,
            'position': n['position']
            
        
        })

In [32]:
max(bet)

0.00113408

In [33]:
# new_nodes[1:10]

In [24]:
# Cleanup edges

edges = clixo['elements']['edges']
new_edges = []


for e in edges:
    d = e['data']
    
    data = {
        'source': id2clixo[d['source']],
        'target': id2clixo[d['target']],
    }
    
    new_edges.append({'data': data})

{'selected': False, 'data': {'is_tree': 'TREE', 'target_original': 'CLIXO:10392', 'shared_interaction': 'default', 'name': '', 'id': '156922', 'target': '18081', 'shared_name': '', 'interaction': 'default', 'SUID': 156922, 'selected': False, 'EdgeBetweenness': 36.0, 'source': '20026', 'source_original': 'CLIXO:9735'}}


In [29]:
clixo_compact = {
    'data': {
        'name': 'CLIXO Tree'
    },
    'elements': {
        'nodes': new_nodes,
        'edges': new_edges
    }
}

with open('./data/clixo-compact.json', 'w') as outfile:
    json.dump(clixo_compact, outfile)

## And same for GO


In [34]:
!wget 'https://gist.githubusercontent.com/keiono/b7c047c1166681ef7170881217819938/raw/b81de1da63075ceb0e4c8d99adeacf83d0193cdb/goWOgenes.cyjs' -O ./data/go-final2.cyjs




--2017-01-10 17:26:02--  https://gist.githubusercontent.com/keiono/b7c047c1166681ef7170881217819938/raw/b81de1da63075ceb0e4c8d99adeacf83d0193cdb/goWOgenes.cyjs
Resolving gist.githubusercontent.com... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to gist.githubusercontent.com|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10378739 (9.9M) [text/plain]
Saving to: ‘./data/go-final2.cyjs’

./data/go-final2.cy 100%[===================>]   9.90M  29.2MB/s    in 0.3s    

2017-01-10 17:26:03 (29.2 MB/s) - ‘./data/go-final2.cyjs’ saved [10378739/10378739]



In [35]:
with open('./data/go-final2.cyjs', 'r') as f:
    gotree = json.load(f)

In [43]:
import pandas as pd

df = pd.read_csv('./data/collapsed_go.no_IGI.propagated.term_sizes', sep='\t', names=['id', 'genes'])
df.head(10)

,id,genes
0,GO:0000001,27
1,GO:0000002,42
2,GO:0000003,448
3,GO:0000006,1
4,GO:0000007,1
5,GO:0000009,9
6,GO:0000010,1
7,GO:0000011,18
8,GO:0000014,7
9,GO:0000015,5


In [103]:
math.exp(max(df['genes'])/1000)

613.3894179106748

In [67]:
go2genes = {}

for row in df.itertuples():
    go2genes[row[1]] = row[2].item()

In [68]:
nodes = gotree['elements']['nodes']
nodes[1]

{'data': {'AverageShortestPathLength': 17.18280147,
  'BetweennessCentrality': 0.00015342,
  'ClosenessCentrality': 0.05819773,
  'ClusteringCoefficient': 0.0,
  'Degree': 2,
  'Eccentricity': 26,
  'EdgeCount': 2,
  'Indegree': 1,
  'IsSingleNode': False,
  'NeighborhoodConnectivity': 2.0,
  'NumberOfDirectedEdges': 2,
  'NumberOfUndirectedEdges': 0,
  'Outdegree': 1,
  'PartnerOfMultiEdgedNodePairs': 0,
  'Radiality': 0.47797415,
  'SUID': 13097,
  'SelfLoops': 0,
  'Stress': 26070,
  'TopologicalCoefficient': 0.5,
  'id': '13097',
  'id_original': 'GO:0016973',
  'name': 'poly(A)+ mRNA export from nucleus',
  'namespace': 'biological_process',
  'selected': False,
  'shared_name': 'poly(A)+ mRNA export from nucleus',
  'type': ''},
 'position': {'x': 15274.211570440326, 'y': 12397.140132129309},
 'selected': False}

In [97]:
id2go = {}
new_nodes = []
import math

for n in nodes:
    d = n['data']
    
    id2go[d['id']] = d['id_original']
    name = d['name']
    genes = go2genes[d['id_original']]
        
    data = {
        'id': d['id_original'],
        'name': name,
        'geneCount': genes,
        'size': math.exp(genes/1000),
        'namespace': d['namespace']
    }
    
    new_nodes.append({
            'data': data,
            'position': n['position']
        })

In [98]:
new_nodes[1]

{'data': {'geneCount': 23,
  'id': 'GO:0016973',
  'name': 'poly(A)+ mRNA export from nucleus',
  'namespace': 'biological_process',
  'size': 1.0232665395472176},
 'position': {'x': 15274.211570440326, 'y': 12397.140132129309}}

In [99]:
math.exp(6000/1000)

403.4287934927351

In [100]:
edges = gotree['elements']['edges']
new_edges = []


for e in edges:
    d = e['data']
    
    data = {
        'source': id2go[d['source']],
        'target': id2go[d['target']],
        'branch': d['branch']
    }
    
    new_edges.append({'data': data})

In [101]:
new_edges[1]

{'data': {'branch': 'BP', 'source': 'GO:0043648', 'target': 'GO:0043650'}}

In [102]:
go_compact = {
    'data': {
        'name': 'GO Tree'
    },
    'elements': {
        'nodes': new_nodes,
        'edges': new_edges
    }
}

with open('./data/go-compact.json', 'w') as outfile:
    json.dump(go_compact, outfile)